In [ ]:
!pip install -q gradio google-generativeai chromadb sentence-transformers tenacity --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0

In [ ]:
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import chromadb
from sentence_transformers import SentenceTransformer
from tenacity import retry, stop_after_attempt, wait_exponential

# Set up Gemini API
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

# Initialize ChromaDB
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="sherlock_knowledge_base")

# Initialize sentence transformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def populate_knowledge_base():
    sherlock_data = [
        "Sherlock Holmes is a fictional detective created by Sir Arthur Conan Doyle.",
        "Holmes is known for his logical reasoning and observational skills.",
        "221B Baker Street is the London address of Sherlock Holmes.",
        "Dr. John Watson is Holmes' friend and assistant.",
        "Sherlock Holmes often uses the phrase 'Elementary, my dear Watson.'",
        "Holmes plays the violin and is known to use cocaine when bored.",
        "The character of Sherlock Holmes was inspired by Dr. Joseph Bell.",
        "Holmes has a brilliant mind and a vast knowledge of criminology.",
        "His arch-nemesis is the criminal mastermind Professor James Moriarty.",
        "Sherlock Holmes first appeared in the novel 'A Study in Scarlet' in 1887.",
        "Holmes is known for his deerstalker cap and Inverness cape, though these were not mentioned in the original stories.",
        "The famous detective has been portrayed in numerous films, TV series, and stage productions.",
        "Sherlock Holmes is known for his use of forensic science in his investigations.",
        "The Holmes stories popularized the use of fingerprint evidence in detective work.",
        "Sherlock has a older brother named Mycroft, who is described as even more intelligent but lacking in energy.",
        "The phrase 'The game is afoot' is often associated with Sherlock Holmes.",
        "Holmes is a master of disguise and often uses this skill in his investigations.",
        "The detective is known for his ability to draw large conclusions from small observations.",
        "Sherlock Holmes has been officially recognized by the Royal Society of Chemistry as an honorary fellow.",
        "The Sherlock Holmes Museum is located at 221B Baker Street in London.",
    ]

    embeddings = embedder.encode(sherlock_data).tolist()

    collection.add(
        documents=sherlock_data,
        embeddings=embeddings,
        ids=[f"sherlock_{i}" for i in range(len(sherlock_data))]
    )

# Populate the knowledge base if it's empty
if collection.count() == 0:
    populate_knowledge_base()


In [ ]:
class SherlockAI:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-pro')
        self.chat = self.model.start_chat(history=[])

    def query_knowledge_base(self, query):
        query_embedding = embedder.encode([query]).tolist()
        results = collection.query(query_embeddings=query_embedding, n_results=3)
        return " ".join(results['documents'][0])

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def get_response(self, user_input):
        context = self.query_knowledge_base(user_input)
        prompt = f"""You are Sherlock Holmes, the brilliant detective. Use the following context and your deductive skills to respond to the user's input. Maintain your character, speaking as Sherlock Holmes would.

Context: {context}

User Input: {user_input}

Your Response:"""

        try:
            response = self.chat.send_message(prompt)
            return response.text
        except Exception as e:
            raise Exception(f"I'm afraid a complication has arisen in our communication: {str(e)}")

    def fallback_response(self, user_input):
        return f"My dear Watson, it seems our usual methods of deduction are temporarily unavailable. However, regarding your query about '{user_input}', I can assure you that with time and careful observation, we shall unravel this mystery. Let us revisit this matter when our resources are fully at our disposal."


In [ ]:
sherlock = SherlockAI()

def chatbot(message, history):
    exit_keywords = ["bye", "goodbye", "exit", "quit", "end"]

    if any(keyword in message.lower() for keyword in exit_keywords):
        return [{"role": "assistant", "content": "Farewell, my dear friend. Until our next adventure! (The program will now exit.)"}]

    try:
        response = sherlock.get_response(message)
        return [{"role": "assistant", "content": response}]
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return [{"role": "assistant", "content": sherlock.fallback_response(message)}]

with gr.Blocks() as demo:
    gr.ChatInterface(
        fn=chatbot,
        title="SherlockAI: The World's Only Consulting Detective",
        description="Engage in a conversation with the brilliant Sherlock Holmes. Ask about his cases, methods, or any topic you'd like him to deduce! Say 'goodbye' to end the conversation.",
        examples=[
            "Tell me about your most famous case.",
            "What's your opinion on modern technology?",
            "How would you solve a cybercrime?",
            "Describe your relationship with Dr. Watson.",
            "What's your method for analyzing complex data?"
        ]
    )

demo.launch(share=True, debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://23499324ee6774aa30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error occurred: RetryError[<Future at 0x78a11144ec50 state=finished raised Exception>]


Error occurred: RetryError[<Future at 0x78a11262be50 state=finished raised Exception>]
